# 2020-04-04 SmLE x SmBRE


## Aim

Redo the analysis with the latest genome and everything


## Environment and data

In [ ]:
conda env create -f .env/env.yml

In [1]:
# Activate the environment
source $(sed "s,/bin/conda,," <<<$CONDA_EXE)/etc/profile.d/conda.sh
conda activate cerc_prod

(cerc_prod) 

: 1

### Genome

In [3]:
gdir="data/genome"
[[ ! -d "$gdir" ]] && mkdir -p "$gdir"

wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz

pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz"

# Preparing indices
bwa index "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
samtools faidx "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"
gatk CreateSequenceDictionary -R "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa"

(PZQ-R) (PZQ-R) (PZQ-R) --2020-05-13 15:22:46--  ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz
           => ‘data/genome/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz’
Resolving ftp.ebi.ac.uk... 193.62.197.74
Connecting to ftp.ebi.ac.uk|193.62.197.74|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577 ... done.
==> SIZE schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz ... 120198881
==> PASV ... done.    ==> RETR schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz ... done.
Length: 120198881 (115M) (unauthoritative)

schistosoma_mansoni 100%[===================>] 114.63M   893KB/s    in 2m 38s  

2020-05-13 15:25:25 (744 KB/s) - ‘data/genome/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa.gz’ saved [1

: 1

### snpEff database

To annotate the functional consequences of the variant in the VCF file, we use the snpEff tool. Because the reference genome is recent, we have to build a snpEff database in order to annotate the variants. 

In [ ]:
# Create snpEff folder
sedir="$gdir/snpEff/"
mkdir "$sedir"

# Download the S. mansoni genome annotation
wget -P "$gdir" ftp://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS14/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz
pigz -d "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3.gz"

# Convert gff to gtf to make it easier for snpEff
gffread "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gff3" -T -o "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gtf"

# Custom snpEff config file
cp ~/local/pckg/python/miniconda3/envs/host_spec/share/snpeff-4.3.1t-3/snpEff.config "$sedir"

## Identify mitochondrial chromosome
mito=$(grep -i ">.*mito.*" "$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" | sed -r "s/>(.*) .*/\1/")

##  Add schistosome genome and specify mitochondrial chromosome
sed -i -r "s|data.dir = ./data/|data.dir = .| ; s|(# Ebola virus)|# S. mansoni\nsmansoni.genome: Schistosoma_mansoni\n\tsmansoni.$mito.codonTable : Trematode_Mitochondrial\n\n\1|" "$sedir"/snpEff.config

# Create the mandatory folder and link to the needed files
mkdir "$sedir"/genomes
ln -s "$PWD/$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.genomic.fa" "$PWD/$sedir/genomes/smansoni.fa"

mkdir "$sedir"/smansoni
ln -s "$PWD/$gdir/schistosoma_mansoni.PRJEA36577.WBPS14.annotations.gtf" "$PWD/$sedir/smansoni/genes.gtf"

# Generate the database
snpEff build -v -c data/genome/snpEff/snpEff.config smansoni

### Library

In [4]:
ldir="data/libraries/"
[[ ! -d "$ldir" ]] && mkdir -p "$ldir"

# Linking F2
data_ln.sh -d ../2016-05-17\ SmLE\ x\ SmBRE/0-Libraries/A -e *F2*fastq.gz -t data/libraries/
data_ln.sh -d ../2016-05-17\ SmLE\ x\ SmBRE/0-Libraries/B -e *F2*fastq.gz -t data/libraries/
data_ln.sh -d ../2016-05-17\ SmLE\ x\ SmBRE/0-Libraries_2/ -e *fastq.gz -t data/libraries/
data_ln.sh -d ../2016-05-17\ SmLE\ x\ SmBRE/0-Libraries_3/B -e *fastq.gz -t data/libraries/

# Linking F1
data_ln.sh -d ../2016-05-17\ SmLE\ x\ SmBRE/0-Libraries/A -e *F1*fastq.gz -t data/libraries/
data_ln.sh -d ../2016-05-17\ SmLE\ x\ SmBRE/0-Libraries/B -e *F1*fastq.gz -t data/libraries/

# Linking parents
data_ln.sh -d ../2016-05-17\ SmLE\ x\ SmBRE/0-Libraries/WGS_parents/ -e *fastq.gz -t data/libraries/

(PZQ-R) (PZQ-R) (PZQ-R) (PZQ-R) 

: 1

### Known variants

List of variants from the SmLE x SmHR cross that segregated in a Mendelian fashion in F1. This list was generated from alignments using the v5 genome. This has been lifted over to the latest genome using the [flo pipeline](https://github.com/wurmlab/flo/tree/727f10b2b1c57a0514835d302d7f6345d3a34ffb).

In [ ]:
# Index file
gatk IndexFeatureFile -I "$gdir/sm_dbSNP_v7.vcf"

## Run snakemake pipeline

In [ ]:
snakemake --snakefile snakefile --cluster "qsub -V -cwd -o status -j y -r y -pe smp 12 -S /bin/bash" --jobs 384 -w 300

## Variant annotation

Functional consequences of variants is annotated to the VCF file using snpEff.

In [ ]:
snpEff -v -c "$sedir/snpEff.config" smansoni "data/calling/cerc_prod.vcf.gz" | bgzip -c -@ $(nproc) > "data/calling/cerc_prod_snpEff.vcf.gz"

# Index the VCF file
tabix "data/calling/cerc_prod_snpEff.vcf.gz"

In [11]:
# Move file in the right folder
mv -t "data/calling/" snpEff_summary.* snpEff_genes.txt

(cerc_prod) (cerc_prod) 

: 1

### Selection of informative sites

Because variant calling was performed on exome capture samples (F1 and F2) and whole genome sequenced samples (parents), most of the site called are uninformative (only identified in parents). To speed up analysis, only sites that have more less than 50% of missing data will be retained.

In [3]:
gatk SelectVariants \
        -V "data/calling/cerc_prod_snpEff.vcf.gz" \
        --max-nocall-fraction 0.5 \
        -O "data/calling/cerc_prod_snpEff_reduced.vcf.gz"

Using GATK jar /master/fcheval/local/pckg/python/miniconda3/envs/cerc_prod/share/gatk4-4.1.7.0-0/gatk-package-4.1.7.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /master/fcheval/local/pckg/python/miniconda3/envs/cerc_prod/share/gatk4-4.1.7.0-0/gatk-package-4.1.7.0-local.jar SelectVariants -V data/calling/cerc_prod_snpEff.vcf.gz --max-nocall-fraction 0.5 -O data/calling/cerc_prod_snpEff_reduced.vcf.gz
23:02:48.844 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/master/fcheval/local/pckg/python/miniconda3/envs/cerc_prod/share/gatk4-4.1.7.0-0/gatk-package-4.1.7.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
Jun 21, 2020 11:02:49 PM shaded.cloud_nio.com.google.auth.oauth2.ComputeEngineCredentials runningOnComputeEngine
INFO: Failed to detect whether we are running on Google Compute Engine.
23:02:49.906 INFO  Select

23:11:00.103 INFO  ProgressMeter -    SM_V7_ZW:22063135              8.2               1266000         155147.1
23:11:10.374 INFO  ProgressMeter -    SM_V7_ZW:25195527              8.3               1293000         155200.0
23:11:20.639 INFO  ProgressMeter -    SM_V7_ZW:28354401              8.5               1319000         155135.1
23:11:30.656 INFO  ProgressMeter -    SM_V7_ZW:31622083              8.7               1345000         155146.7
23:11:40.864 INFO  ProgressMeter -    SM_V7_ZW:35260079              8.8               1372000         155215.0
23:11:51.232 INFO  ProgressMeter -    SM_V7_ZW:39297817              9.0               1399000         155234.9
23:12:01.385 INFO  ProgressMeter -    SM_V7_ZW:42908389              9.2               1426000         155314.6
23:12:11.709 INFO  ProgressMeter -    SM_V7_ZW:45923529              9.4               1453000         155344.0
23:12:21.839 INFO  ProgressMeter -    SM_V7_ZW:48278818              9.5               1479000         1

23:23:34.856 INFO  ProgressMeter -      SM_V7_4:1455892             20.7               3196000         154104.3
23:23:45.185 INFO  ProgressMeter -      SM_V7_4:3765168             20.9               3223000         154126.7
23:23:55.485 INFO  ProgressMeter -      SM_V7_4:5796481             21.1               3250000         154152.5
23:24:05.568 INFO  ProgressMeter -      SM_V7_4:8336418             21.3               3275000         154109.8
23:24:15.778 INFO  ProgressMeter -     SM_V7_4:10930346             21.4               3302000         154146.0
23:24:26.083 INFO  ProgressMeter -     SM_V7_4:13393724             21.6               3329000         154170.3
23:24:36.104 INFO  ProgressMeter -     SM_V7_4:14946468             21.8               3354000         154135.9
23:24:46.481 INFO  ProgressMeter -     SM_V7_4:17419630             21.9               3382000         154197.2
23:24:56.766 INFO  ProgressMeter -     SM_V7_4:19985892             22.1               3408000         1

: 1